# FYS-STK4155 – Project 1: Regression analysis and resampling methods

##### Authors: Adele Zaini, Gaute Arnesson Holen, Fridtjof Gjengset
##### Date: 11. October, 2021

## Before we start: 

### General introduction

In this first project we are asked to explore the techniques of Regression, as the basic step to go further deeply into Machine Learning, and the resampling methods, such as Bootstrap and Cross-validation.

We started working with a vanilla dataset given by the Franke function, depending of two indenpendent variables $x$ and $y$, in order to set up all our regression and resampling methods. At the end, we performed the code with a real dataset taken from a terrain survey.

Franke function is a weighted sum of four exponentials:

$$
\begin{align*}
f(x,y) &= \frac{3}{4}\exp{\left(-\frac{(9x-2)^2}{4} - \frac{(9y-2)^2}{4}\right)}+\frac{3}{4}\exp{\left(-\frac{(9x+1)^2}{49}- \frac{(9y+1)}{10}\right)} \\
&+\frac{1}{2}\exp{\left(-\frac{(9x-7)^2}{4} - \frac{(9y-3)^2}{4}\right)} -\frac{1}{5}\exp{\left(-(9x-4)^2 - (9y-7)^2\right) }.
\end{align*}
$$

The function is defined for $x,y\in [0,1]$.

The Regression methods that has been implemented are the Linear (Ordinary Least Square), Ridge and Lasso Regression. For further details, see the introduction of the respective exercise.

To access the quality of the model, we perfom an error analysis, usually involving the Mean Square Error ($MSE$) and the $R^2$ score function:

$$
MSE(\hat{y},\hat{\tilde{y}}) = \frac{1}{n}
\sum_{i=0}^{n-1}(y_i-\tilde{y}_i)^2,
$$

$$
R^2(\hat{y}, \tilde{\hat{y}}) = 1 - \frac{\sum_{i=0}^{n - 1} (y_i - \tilde{y}_i)^2}{\sum_{i=0}^{n - 1} (y_i - \bar{y})^2},
$$

where $\tilde{\hat{y}}_i$ is the predicted value of the $i-th$ sample, $y_i$ is the corresponding true value and $\hat{y}$ is the mean value given by $ \bar{y} =  \frac{1}{n} \sum_{i=0}^{n - 1} y_i$.

The $MSE$ is explored as function of complexity, in particular when dividing the dataset into training and test data, in order to find the optimal "complexity" (i.e. degree of polynomial). To get a better extimation of the $MSE$ it has been coupled with resampling methods, and to have a better understanding and deeper comprehension, a bias-variance tradeoff analysis has been performed.


### Code structure

The code is structured into libraries (e.g. `regression.py`), which gather all the functions needed for solving the various tasks, while the specific task codes are in `task*.py` files.

In the course of this notebook, we inserted runnable cells to run the task codes directly and dynamically from here to show up the results.

After a brief introduction, we will explain the algorithm, show the results and comment on them.

*Note: a further improvement to the code would be structuring as object-oriented. We have started converting the code in a new branch.*


## Exercise 1: Ordinary Least Square (OLS) on the Franke function

### Introduction

### Algorithm

After setting up our vanilla dataset using the FrankeFunction with two indipendent variables, we perfom a Ordinary Least Square regression.
We structured the code with all the useful functions in a library called 'linear_regression.py', while specific analysises due for the exercise are in 'task1.py'.

The implemented functions are the following:

- **create_X**(x,y,n) : Creates the designmatrix with a complexity n.
- **Frankefunction**(x,y) : Calculates the frankefunction fro the values x and y.
- **Plot_frankefunction**(z,y,z) : Creates a 3d plot
- **Split_and_Scale**(X,z,test_size, scale) : Splits the dataset using **train_test_split** from scikit learn, into a test_size which has default value set to *0.2*. Scales the data using **StandardScalar()** if boolean argument *scale=True*(it's default value). 
- **OLS_solver**(X_train, X_test, z_train, z_test) : Which calculates the optimal $\hat{\beta}$ values through matix inversion and returns these values, as well as our model $\tilde{z}$ and out prediction. 
- **MSE**(y_data, y_model) : calculates the mean square error of a dataset compared to a model or prediction.
- **R2**(y_data,y_model) : calculates the R2 score of a dataset compared to a model or prediction.

After splitting the data using **Split_and_Scale()**, we find the OLS using **OLS_solver()** on the Frankefunction by matrix inversion. 


We start by simply importing the functions and creating the dataset, as well as adding some noise to the dataset:


In [ ]:
import numpy as np
from random import random, seed
from linear_regression import FrankeFunction, create_X, Split_and_Scale, OLS_solver, MSE, R2, Plot_FrankeFunction

# Create vanilla dataset:
np.random.seed(1233)

n = 25

x = np.linspace(0,1,n)
y = np.linspace(0,1,n) 
x, y = np.meshgrid(x,y)

sigma_N = 0.1; mu_N = 0 
z = FrankeFunction(x,y) +mu_N+sigma_N*np.random.randn(n,n)

Plot_FrankeFunction(x,y,z, title="Original noisy dataset")


Above, we see a plot of the frankefunction with some added noise. Having $\sigma = 0.1$ seems to be a reasonable amount of noise, by looking at the plot. 

We can know create the designmatrix, for a polynomial up to the 5th order. We also want to split and scale the dataset, so we can make our model and prediction.

We do not **have** to scale this dataset as we're not working with a lot of different units. If we imagine this to be terrain data, all the data in the set could be mesured in meteres. However, we use boolean as an argument to quickly be able to enable or disable scaling. We decided to scale the data for more adaptive code that can be reused for other purposes(Adele agree?).

To create the designmatrix, as well as to split and scale out data, we run the following lines:

In [ ]:
degree=5

X = create_X(x, y, degree)
X_train, X_test, z_train, z_test = Split_and_Scale(X,np.ravel(z)) #StardardScaler, test_size=0.2, scale=true

As we now have our training and test data seperated we can go on to find the optimal $\hat{\beta}$ values for the training set, as well as our model and prediction. We do this using **OLS_solver()** which calculates $\hat{\beta}$ through matrix inversion:

$$\hat{\beta}^{OLS} = (X^T X)^{-1}X^T z$$

Where X is the designmatrix for the traindata and z is the training datapoints from the frankefunction. To avoid problems arising from singular matrices, we use the **pinv()** function to find the psudoinverse.

This yields both our model and the prediction by
$$\tilde{z} = X \hat{\beta}$$
Where X is the designmatrix for the training data to produce our model, and the designmatrix for our test data to produce out prediction. 

We run the following line to find our result:

In [ ]:
ols_beta, z_tilde,z_predict = OLS_solver(X_train, X_test, z_train, z_test)

As we now have our model and prediction we can go on to calculate the mean square error and the R2 score for both our model and presdiction:

Finally, we know that the confidence interval of beta can be found by:
$$var(\beta) = \sigma^2 (X^T X)^{-1}$$
Where $\sigma$ si our deviation from the dataset, and X is the designmatrix for the training data.

We can therefore calculate the confidence interval in the following way:


In [ ]:
%run ../1project/task1.py

Above we can see the confidence interval for our estimators with a 95% certainty. 


## Exercise 2:

We can reproduce figure. 2.11 from Hastie, Tibshiani and Friedman, by plotting the MSE as a function of the complexity of the model. By using the same x, y and z values, we calculate the designmatrix **X** with a complexity from n=2 to n=20, where n is the compexity/degree og the polynomial. For each designmtrix we find the OLS and calcualte the MSE for the test and training data. We import the function **plot_ols_complexity**(x, y, z, complexity) from the linear_regression library, which does exactly this. It's default valye for complexity is set to **range(2,20)**. After importing **plot_ols_complexity()** we can therefore simply write:



In [ ]:
from linear_regression import plot_ols_complexity
complexity = range(2,20)
plot_ols_complexity(x,y,z,complexity)

Initially the MSE for both test and train decreases as out model imporves with the complexity of the polynomial. Around a polynomial of the 5th degree, we can see that the MSE for the test data increases, while the MSE for the train data continues to decrease. This is beacuse the increased complexity of the designmatrix causes overfitting. Out model therefore fits the training data better, but gives a poor model for the test data, because it has overfitted to the training set. It seems out model does best around a complexity of n=5.


In [ ]:
%run ../1project/task2.py

## Exercise 3:

In [ ]:
%run ../1project/task3.py

## Exercise 4:

## Exercise 5:

## Exercise 6:

In [ ]:
import pandas as pd
# Confidence interval
beta_ols_variance = sigma_N**2 * np.linalg.pinv(X_train.T @ X_train) #Calculates variance of beta
var_diag=np.diag(beta_ols_variance)
ci1 = ols_beta - 1.96 * np.sqrt(var_diag)/(X.shape[0])
ci2 = ols_beta + 1.96 * np.sqrt(var_diag)/(X.shape[0])
print('Confidence interval of β-estimator at 95 %:')
ci_df = {r'$β_{-}$': ci1,
         r'$β_{ols}$': ols_beta,
         r'$β_{+}$': ci2}
ci_df = pd.DataFrame(ci_df)
display(np.round(ci_df,3))#prec
print("––––––––––––––––––––––––––––––––––––––––––––")
